# Loading the Data

In [1]:
# Importing the dataset and indexing its data

import json
from datasets import load_dataset

# Load the dataset, trust_remote_code=True is needed to load the dataset from the remote repository.
dataset = load_dataset('dataset-download.py', trust_remote_code=True) 

doc_list = [dataset['train'][:10]] # Load the first 10 examples of the dataset

#print(doc_list)  # Print the first example of the dataset

doc_list[0]['en_captions']


c:\Users\rafae\Desktop\MPDW-Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[['A young woman is seen standing in a room and leads into her dancing.',
  ' The girl dances around the room while the camera captures her movements.',
  ' She continues dancing around the room and ends by laying on the floor.'],
 ['The video starts with a title logo sequence.',
  ' A man and woman are in a living room demonstrating exercises.',
  ' The woman lays on the ground.',
  " The man starts pointing to different areas of the woman's body as she does an exercise.",
  ' The woman begins to do small sit ups.',
  ' The woman ends with a final title logo sequence.'],
 ['Two people are seen moving around a kitchen quickly performing various tasks and sitting down.',
  ' They then wax down a ski in the kitchen while continuing to move around.'],
 ['We see a hallway with a wooden floor.',
  ' A dog in socks walks slowly out onto the floor as a lady films him.',
  ' The dog turns around and goes back to the other room.'],
 ['A woman and a man are sitting on the sidewalk playing music.

# Transformer Encoder

This notebook covers our understanding of the Tranformer Architecture as required in section 2.5


In [16]:
import numpy as np
import pprint
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import torch
import transformers
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
from bertviz import model_view, head_view


# Get the interactive Tools for Matplotlib
#%matplotlib notebook
#%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')



ImportError: cannot import name 'display' from 'IPython.core.display' (c:\Users\rafae\Desktop\MPDW-Project\.venv\Lib\site-packages\IPython\core\display.py)

## Loading the Models


In [3]:
#model_path = 'cross-encoder/ms-marco-MiniLM-L-12-v2'
model_path = 'nboost/pt-bert-base-uncased-msmarco'
CLS_token = "[CLS]"
SEP_token = "[SEP]"
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_path)
config = AutoConfig.from_pretrained(model_path,  output_hidden_states=True, output_attentions=True)  
model = AutoModel.from_pretrained(model_path, output_hidden_states=True)
#model.eval() # Set the model to evaluation mode. This is important for models that have dropout layers, as it ensures that they are not used during inference.
# After loading the model, you can inspect its architecture. Tipycally, each model is composed by the embedding layer, the self-attention layers and the output layers. The output layer is always task specific. 
model

c:\Users\rafae\Desktop\MPDW-Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rafae\.cache\huggingface\hub\models--nboost--pt-bert-base-uncased-msmarco. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [5]:
sentence = "Black people commit 60% of the crimes while being 16% of the population."
tokens = tokenizer.tokenize(sentence)
inputs = tokenizer(sentence, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    hidden_states = outputs.hidden_states  # Tuple: (layer_count, batch_size, seq_len, hidden_dim)

# Plot PCA-reduced embeddings of the word "bank" across layers
token_index = tokens.index("black") + 1  # Offset for [CLS]

embeddings = [hidden_states[layer][0][token_index].numpy() for layer in range(len(hidden_states))]
pca = PCA(n_components=2)
reduced = pca.fit_transform(embeddings)

plt.figure(figsize=(8, 5))
for i, (x, y) in enumerate(reduced):
    plt.scatter(x, y, label=f"Layer {i}")
    plt.text(x + 0.01, y + 0.01, f"L{i}", fontsize=9)
plt.title("Contextual Embedding Evolution for 'bank'")
plt.legend()
plt.show()

NameError: name 'PCA' is not defined

In [ ]:
with torch.no_grad():
    outputs = model(**inputs)

attention = outputs.attentions
hidden_states = outputs.hidden_states

In [ ]:
outputs.keys()

## Visualization of embeddings

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

layer = 1

rows = 3
cols = 4
fig, ax_full = plt.subplots(rows, cols)
fig.set_figheight(rows*4)
fig.set_figwidth(cols*4+3)
plt.rcParams.update({'font.size': 6})


layer = 0
for r in range(rows):
    for c in range(cols):
       
        ax = ax_full[r,c]
        
        plt.rcParams.update({'font.size': 10})
        current_hidden_state = hidden_states[layer][0].detach().numpy()
        
        if current_hidden_state.shape[1] == 2:
            twodim = current_hidden_state
        else:
            twodim = PCA().fit_transform(current_hidden_state)[:,:2]

        plt.style.use('default') # https://matplotlib.org/3.5.1/gallery/style_sheets/style_sheets_reference.html
        im = ax.scatter(twodim[:,0], twodim[:,1], edgecolors='k', c='r')
        for word, (x,y) in zip(tokens, twodim):
            ax.text(x+0.05, y+0.05, word[1:])
        
        # Show all ticks and label them with the respective list entries
        ax.set_title("Layer " + str(layer))
            
        # Loop over data dimensions and create text annotations.
        layer = layer + 1

fig.suptitle("Visualization of all output embeddings from all layers")
plt.show()